# Regressiemodellen

In [1]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import pyodbc
import matplotlib.pyplot as plt

# Database connectie
De data komt uit de datawarehouse.

In [2]:
# database name
DB = {
    'servername': '(local)\\SQLEXPRESS',
    'database': 'DEDS_DataWarehouse'}

In [3]:
export_conn = pyodbc.connect('DRIVER={SQL Server};SERVER=' + DB['servername'] + ';DATABASE=' + DB['database'] + ';Trusted_Connection=yes')


# Create a cursor from the connection
export_cursor = export_conn.cursor()

# check if connection is successful, else throw an error
if export_conn:
    print("Connection with database is established")
else:
    print("Connection with database is not established")
    raise Exception("Connection with database is not established")

Connection with database is established


# Tabellen inlezen
We hebben de order_header, order_details en order tabellen nodig

In [4]:
order_query = "SELECT * FROM [Order]"

order_result = export_cursor.execute(order_query)
order_fetch = order_result.fetchall()
order_columns = [column[0] for column in order_result.description]
order = pd.DataFrame.from_records(order_fetch, columns=order_columns)

# dropping current value and last updated columns
order = order.drop(columns=['CURRENT_VALUE', 'LAST_UPDATED'])

order.head()

,ORDER_SK,ORDER_order_number,ORDER_ORDER_METHOD_CODE_method_code,ORDER_ORDER_METHOD_EN_method
0,1,1153,7,Sales visit
1,2,1154,4,E-mail
2,3,1155,7,Sales visit
3,4,1156,5,Web
4,5,1157,2,Telephone


In [5]:
order_header_query = "SELECT * FROM Order_header"

order_header_result = export_cursor.execute(order_header_query)
order_header_fetch = order_header_result.fetchall()
order_header_columns = [column[0] for column in order_header_result.description]
order_header = pd.DataFrame.from_records(order_header_fetch, columns=order_header_columns)

# dropping current value and last updated columns
order_header = order_header.drop(columns=['CURRENT_VALUE', 'LAST_UPDATED'])

order_header.head()

,ORDER_HEADER_SK,ORDER_HEADER_number,ORDER_HEADER_RETAILER_CODE,ORDER_HEADER_SALES_STAFF_CODE,ORDER_HEADER_SALES_BRANCH_CODE,ORDER_HEADER_ORDER_DATE,ORDER_HEADER_RETAILER_SITE_CODE,ORDER_HEADER_RETAILER_CONTACT_CODE,ORDER_HEADER_ORDER_order_number
0,1,1153,100,54,5,2020-04-14,285,349,1
1,2,1154,100,54,5,2020-10-01,285,349,2
2,3,1155,100,52,5,2020-04-21,340,355,3
3,4,1156,100,52,5,2020-09-09,340,355,4
4,5,1157,100,54,5,2020-04-10,351,356,5


In [6]:
order_details_query = "SELECT * FROM Order_details"

order_details_result = export_cursor.execute(order_details_query)
order_details_fetch = order_details_result.fetchall()
order_details_columns = [column[0] for column in order_details_result.description]
order_details = pd.DataFrame.from_records(order_details_fetch, columns=order_details_columns)

# dropping current value and last updated columns
order_details = order_details.drop(columns=['CURRENT_VALUE', 'LAST_UPDATED'])

order_details.head()

,ORDER_DETAILS_SK,ORDER_DETAILS_code,ORDER_DETAILS_QUANTITY_quantity,ORDER_DETAILS_TOTAL_COST_total,ORDER_DETAILS_TOTAL_MARGIN_margin,ORDER_DETAILS_RETURN_CODE_returned,ORDER_DETAILS_ORDER_NUMBER_order,ORDER_DETAILS_PRODUCT_NUMBER_product,ORDER_DETAILS_UNIT_ID_unit
0,1,100146,16,8072.6400,9496.0000,1975,4408,23,144
1,2,100149,64,65.2800,131.2000,1976,5013,24,147
2,3,100199,22,149.1600,298.5400,1979,5010,40,197
3,4,100250,52,6578.5200,9868.0400,2298,5135,54,248
4,5,100641,6,491.2200,1080.7800,2301,5150,15,635


In [7]:
unit_query = "SELECT * FROM Unit"

unit_result = export_cursor.execute(unit_query)
unit_fetch = unit_result.fetchall()
unit_columns = [column[0] for column in unit_result.description]
unit = pd.DataFrame.from_records(unit_fetch, columns=unit_columns)

# dropping current value and last updated columns
unit = unit.drop(columns=['CURRENT_VALUE', 'LAST_UPDATED'])

unit.head()

,UNIT_SK,UNIT_id,UNIT_COST_cost,UNIT_PRICE_price,UNIT_SALE_sale
0,1,1,16.1000,22.5400,22.5400
1,2,2,16.1000,22.5400,22.5400
2,3,3,16.1000,22.5400,22.5400
3,4,4,16.1000,22.5400,22.5400
4,5,5,16.1000,22.5400,22.5400


In [8]:
product_query = "SELECT * FROM Product"

product_result = export_cursor.execute(product_query)
product_fetch = product_result.fetchall()
product_columns = [column[0] for column in product_result.description]
product = pd.DataFrame.from_records(product_fetch, columns=product_columns)

# dropping current value and last updated columns
product = product.drop(columns=['CURRENT_VALUE', 'LAST_UPDATED'])

product.head()

,PRODUCT_SK,PRODUCT_number,PRODUCT_name_product,PRODUCT_description_description,PRODUCT_image_image,PRODUCT_INTRODUCTION_DATE_introduced,PRODUCT_PRODUCTION_COST_cost,PRODUCT_MARGIN_margin,PRODUCT_LANGUAGE_language,PRODUCT_MINIMUM_SALE_PRICE_minPrice,PRODUCT_PRODUCT_LINE_code,PRODUCT_PRODUCT_LINE_code_en,PRODUCT_PRODUCT_TYPE_code,PRODUCT_PRODUCT_TYPE_code_en
0,1,1,TrailChef Water Bag,"Lightweight, collapsible bag to carry liquids ...",P01CE1CG1.jpg,2011-02-15,4.0000,0.3300,EN,4.3300,1,Camping Equipment,1,Cooking Gear
1,2,10,TrailChef Utensils,"Spoon, fork and knife set made of a light yet ...",P10CE1CG1.jpg,2011-02-15,10.0000,0.4000,EN,10.4000,1,Camping Equipment,1,Cooking Gear
2,3,100,Insect Bite Relief,The Insect Bite Relief helps the itching and s...,P100OP4FA17.jpg,2011-02-15,3.0000,0.5000,EN,3.5000,4,Outdoor Protection,17,First Aid
3,4,101,Hailstorm Steel Irons,Iron is 17-4 stainless steel. Shafts are grap...,P101GE5IR18.jpg,2019-12-15,305.5400,0.4300,EN,305.9700,5,Golf Equipment,18,Irons
4,5,102,Hailstorm Titanium Irons,Made entirely of pure titanium. The ultimate i...,P102GE5IR18.jpg,2019-10-12,380.9500,0.5100,EN,381.4600,5,Golf Equipment,18,Irons


## De tabellen samenvoegen tot 1 dataframe

In [9]:
order_combined = pd.merge(order_header, order, left_on='ORDER_HEADER_ORDER_order_number', right_on='ORDER_SK')
order_combined = pd.merge(order_combined, order_details, left_on='ORDER_SK', right_on='ORDER_DETAILS_ORDER_NUMBER_order')
order_combined = pd.merge(order_combined, unit, left_on='ORDER_DETAILS_UNIT_ID_unit', right_on='UNIT_SK')
order_combined = pd.merge(order_combined, product, left_on='ORDER_DETAILS_PRODUCT_NUMBER_product', right_on='PRODUCT_SK')

order_combined.head()

,ORDER_HEADER_SK,ORDER_HEADER_number,ORDER_HEADER_RETAILER_CODE,ORDER_HEADER_SALES_STAFF_CODE,ORDER_HEADER_SALES_BRANCH_CODE,ORDER_HEADER_ORDER_DATE,ORDER_HEADER_RETAILER_SITE_CODE,ORDER_HEADER_RETAILER_CONTACT_CODE,ORDER_HEADER_ORDER_order_number,ORDER_SK,...,PRODUCT_image_image,PRODUCT_INTRODUCTION_DATE_introduced,PRODUCT_PRODUCTION_COST_cost,PRODUCT_MARGIN_margin,PRODUCT_LANGUAGE_language,PRODUCT_MINIMUM_SALE_PRICE_minPrice,PRODUCT_PRODUCT_LINE_code,PRODUCT_PRODUCT_LINE_code_en,PRODUCT_PRODUCT_TYPE_code,PRODUCT_PRODUCT_TYPE_code_en
0,8,1160,102,55,5,2020-09-28,381,358,8,8,...,P70PA3EW11.jpg,2013-05-03,80.0000,0.3300,EN,80.3300,3,Personal Accessories,11,Eyewear
1,14,1167,105,54,5,2020-03-14,77,364,14,14,...,P04CE1CG1.jpg,2011-02-15,5.0000,0.2800,EN,5.2800,1,Camping Equipment,1,Cooking Gear
2,49,1207,107,49,4,2020-04-04,173,208,49,49,...,P63PA3WT10.jpg,2011-02-15,30.0000,0.3300,EN,30.3300,3,Personal Accessories,10,Watches
3,59,1217,102,47,4,2021-03-26,13,128,59,59,...,P65PA3WT10.jpg,2013-05-03,39.0000,0.5000,EN,39.5000,3,Personal Accessories,10,Watches
4,61,1219,102,46,4,2021-03-29,24,359,61,61,...,P29CE1PK4.jpg,2013-05-03,41.1800,0.4000,EN,41.5800,1,Camping Equipment,4,Packs


# Data voorbereiden
Om de verkopen te kunnen voorspellen, moeten we de data voorbereiden. We willen de verkopen voorspellen op basis van de orderdatum en een aantal orderdetails.

In [10]:
# removing the sk columns
sk_columns = order_combined.filter(like='SK').columns
order_combined.drop(columns=sk_columns, inplace=True)

# removing columns we won't need
remove_columns = ['ORDER_HEADER_number', 'ORDER_HEADER_RETAILER_CODE', 'ORDER_HEADER_SALES_STAFF_CODE', 'ORDER_HEADER_SALES_BRANCH_CODE', 'ORDER_HEADER_RETAILER_SITE_CODE', 'ORDER_HEADER_RETAILER_CONTACT_CODE', 'ORDER_HEADER_ORDER_order_number', 'ORDER_order_number', 'ORDER_ORDER_METHOD_EN_method', 'ORDER_DETAILS_code', 'ORDER_DETAILS_RETURN_CODE_returned', 'ORDER_DETAILS_ORDER_NUMBER_order', 'ORDER_DETAILS_PRODUCT_NUMBER_product', 'ORDER_DETAILS_UNIT_ID_unit', 'UNIT_id', 'PRODUCT_image_image', 'PRODUCT_PRODUCT_LINE_code_en', 'PRODUCT_PRODUCT_TYPE_code_en', 'PRODUCT_name_product', 'PRODUCT_description_description', 'PRODUCT_LANGUAGE_language']

order_combined.drop(columns=remove_columns, inplace=True)

order_combined.head()

,ORDER_HEADER_ORDER_DATE,ORDER_ORDER_METHOD_CODE_method_code,ORDER_DETAILS_QUANTITY_quantity,ORDER_DETAILS_TOTAL_COST_total,ORDER_DETAILS_TOTAL_MARGIN_margin,UNIT_COST_cost,UNIT_PRICE_price,UNIT_SALE_sale,PRODUCT_number,PRODUCT_INTRODUCTION_DATE_introduced,PRODUCT_PRODUCTION_COST_cost,PRODUCT_MARGIN_margin,PRODUCT_MINIMUM_SALE_PRICE_minPrice,PRODUCT_PRODUCT_LINE_code,PRODUCT_PRODUCT_TYPE_code
0,2020-09-28,5,24,1963.2000,2944.8000,81.8000,122.7000,122.7000,70,2013-05-03,80.0000,0.3300,80.3300,3,11
1,2020-03-14,7,84,439.3200,571.2000,5.2300,7.3200,6.8000,4,2011-02-15,5.0000,0.2800,5.2800,1,1
2,2020-04-04,2,68,2216.1200,3323.8400,32.5900,48.8800,48.8800,63,2011-02-15,30.0000,0.3300,30.3300,3,10
3,2021-03-26,4,40,1560.0000,3120.0000,39.0000,78.0000,78.0000,65,2013-05-03,39.0000,0.5000,39.5000,3,10
4,2021-03-29,7,152,6349.0400,10795.0400,41.7700,71.0200,71.0200,29,2013-05-03,41.1800,0.4000,41.5800,1,4


In [11]:
# converting the columns to the right data types
order_combined['ORDER_HEADER_ORDER_DATE'] = pd.to_datetime(order_combined['ORDER_HEADER_ORDER_DATE'], format='%Y-%m-%d')
order_combined['ORDER_ORDER_METHOD_CODE_method_code'] = order_combined['ORDER_DETAILS_QUANTITY_quantity'].astype(int)
order_combined['ORDER_DETAILS_QUANTITY_quantity'] = order_combined['ORDER_DETAILS_QUANTITY_quantity'].astype(int)
order_combined['ORDER_DETAILS_TOTAL_COST_total'] = order_combined['ORDER_DETAILS_TOTAL_COST_total'].astype(float)
order_combined['ORDER_DETAILS_TOTAL_MARGIN_margin'] = order_combined['ORDER_DETAILS_TOTAL_MARGIN_margin'].astype(float)
order_combined['UNIT_COST_cost'] = order_combined['UNIT_COST_cost'].astype(float)
order_combined['UNIT_PRICE_price'] = order_combined['UNIT_PRICE_price'].astype(float)
order_combined['UNIT_SALE_sale'] = order_combined['UNIT_SALE_sale'].astype(float)
order_combined[ 'PRODUCT_number'] = order_combined[ 'PRODUCT_number'].astype(int)
order_combined['PRODUCT_INTRODUCTION_DATE_introduced'] = pd.to_datetime(order_combined['PRODUCT_INTRODUCTION_DATE_introduced'], format='%Y-%m-%d')
order_combined['PRODUCT_PRODUCTION_COST_cost'] = order_combined['PRODUCT_PRODUCTION_COST_cost'].astype(float)
order_combined['PRODUCT_MARGIN_margin'] = order_combined['PRODUCT_MARGIN_margin'].astype(float)
order_combined['PRODUCT_MINIMUM_SALE_PRICE_minPrice'] = order_combined['PRODUCT_MINIMUM_SALE_PRICE_minPrice'].astype(float)
order_combined['PRODUCT_PRODUCT_LINE_code'] = order_combined['PRODUCT_PRODUCT_LINE_code'].astype(int)
order_combined['PRODUCT_PRODUCT_TYPE_code'] = order_combined['PRODUCT_PRODUCT_TYPE_code'].astype(int)

# Convert datetime objects to numeric representation
order_combined['ORDER_HEADER_ORDER_DATE'] = order_combined['ORDER_HEADER_ORDER_DATE'].apply(lambda x: x.toordinal())
order_combined['PRODUCT_INTRODUCTION_DATE_introduced'] = order_combined['PRODUCT_INTRODUCTION_DATE_introduced'].apply(lambda x: x.toordinal())

# Voorspellen
We willen de ORDER_DETAILS_QUANTITY_quantity voorspellen op basis van de andere kolommen.

In [12]:
# split the data
x = order_combined.drop(columns=['ORDER_DETAILS_QUANTITY_quantity'], axis=1)
y = order_combined['ORDER_DETAILS_QUANTITY_quantity']

# split the data into training and testing data. Randomly so it changes every time
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)

## Regressie model trainen

In [13]:
# create the model
model = LinearRegression()
model.fit(x_train, y_train)

# predict the test data
y_pred = model.predict(x_test)

In [14]:
# Evaluate the model
mse = mean_squared_error(y_test, y_pred)

print('Mean Squared Error:', mse)

Mean Squared Error: 1.997879846026993e-24
